## 2021: Week 25 - The Worst Pokémon

Often our stakeholders can have very niche knowledge and their requests may baffle us. Luckily, as data preppers we have the tools to tackle any dataset, no matter how bizarre. Yes, Carl and Tom have allowed me to create another Pokémon challenge!

The idea came from a YouTube video that I stumbled across: Who Is Pokemon’s LEAST Favorite Pokémon? The logical steps applied in the video felt like they were screaming out for a Preppin' Data challenge to verify the results! But be warned, the answer to this challenge will differ from the conclusion of the video, due to differing datasets.

### Input
We have multiple inputs for this challenge:
1. Gen 1 Pokémon (from Pokémon Database)
2. Evolution Group (from Bulbapedia - also see Preppin' Data 2021 week 10) 
3. Evolutions (Bulbapedia)
4. Mega Evolutions (Pokémon DB)
5. Alolan Pokémon (Pokémon DB)
6. Galarian Pokémon (Pokémon DB)
7. Gigantamax Pokémon (from IGN)
8. Unattainable Pokémon in Sword & Shield (Pokémon DB)
9. Anime appearances for Pokémon (First 116 episodes webscraped from Bulbapedia)

### Challenge
Remember: once a Pokémon meets a condition, their whole evolution group is excluded from consideration. For example, since there is a Mega Beedrill, Weedle and Kakuna cannot be the worst Pokémon since they all belong to the Weedle evolution group.

- Input the data
- Clean up the list of Gen 1 Pokémon so we have 1 row per Pokémon
- Clean up the Evolution Group input so that we can join it to the Gen 1 list 
    - Filter out Starter and Legendary Pokémon
- Using the Evolutions input, exclude any Pokémon that evolves from a Pokémon that is not part of Gen 1 or can evolve into a Pokémon outside of Gen 1
- Exclude any Pokémon with a mega evolution, Alolan, Galarian or Gigantamax form
- It's not possible to catch certain Pokémon in the most recent games. These are the only ones we will consider from this point on
- We're left with 10 evolution groups. Rank them in ascending order of how many times they've appeared in the anime to see who the worst Pokémon is!
- Output the data

###